In [6]:
from collections import defaultdict
from datetime import datetime
from math import sqrt

import numpy as np
import pandas as pd

%matplotlib inline

# Read data

In [10]:
EUCLIDEAN = 'euclidean'
MANHATTAN = 'manhattan'
PEARSON = 'pearson'


def read_ratings_df():
    date_parser = lambda time_in_secs: datetime.utcfromtimestamp(float(time_in_secs))
    return pd.read_csv('ml-latest-small/ratings.csv', parse_dates=['timestamp'], date_parser=date_parser)


class MovieData(object):
    def __init__(self):
        self.ratings_df = read_ratings_df()
        self.ratings = defaultdict(dict)
        self.init_ratings()

    def init_ratings(self):
        for _, row in self.ratings_df.iterrows():
            self.ratings[row['userId']][row['movieId']] = row

    def get_movies(self, user_id):
        return set(self.ratings[user_id].keys())

    def get_unique_user_ids(self):
        return self.ratings_df['userId'].unique()

    def get_shared_ratings(self, user1_id, user2_id):
        movies1 = self.get_movies(user1_id)
        movies2 = self.get_movies(user2_id)

        shared_movies = movies1 & movies2

        ratings = {}

        for movie_id in shared_movies:
            ratings[movie_id] = (
                self.ratings[user1_id][movie_id]['rating'],
                self.ratings[user2_id][movie_id]['rating'],
            )

        return ratings

    @staticmethod
    def shared_ratings_to_np_arrays(shared_ratings):
        return np.array(shared_ratings.values()).T

    def get_euclidean_distance(self, user1_id, user2_id):
        shared_ratings = self.get_shared_ratings(user1_id, user2_id)

        if len(shared_ratings) == 0:
            return 0

        ratings = self.shared_ratings_to_np_arrays(shared_ratings)

        ratings1 = ratings[0]
        ratings2 = ratings[1]

        sum_of_squares = np.power(ratings1 - ratings2, 2).sum()

        return 1 / (1 + sqrt(sum_of_squares))

    def get_manhattan_distance(self, user1_id, user2_id):
        shared_ratings = self.get_shared_ratings(user1_id, user2_id)

        if len(shared_ratings) == 0:
            return 0

        ratings = self.shared_ratings_to_np_arrays(shared_ratings)

        ratings1 = ratings[0]
        ratings2 = ratings[1]

        manhattan_sum = np.abs(ratings1 - ratings2).sum()

        return 1 / (1 + manhattan_sum)

    def get_pearson_correlation(self, user1_id, user2_id):
        shared_ratings = self.get_shared_ratings(user1_id, user2_id)

        num_ratings = len(shared_ratings)

        if num_ratings == 0:
            return 0

        ratings = self.shared_ratings_to_np_arrays(shared_ratings)

        ratings1 = ratings[0]
        ratings2 = ratings[1]

        mean1 = ratings1.mean()
        mean2 = ratings2.mean()

        std1 = ratings1.std()
        std2 = ratings2.std()

        if std1 == 0 or std2 == 0:
            return 0

        std_scores_1 = (ratings1 - mean1) / std1
        std_scores_2 = (ratings2 - mean2) / std2

        # numerically stable calculation of the Pearson correlation coefficient

        return abs((std_scores_1 * std_scores_2).sum() / (num_ratings - 1))

    def get_similar_users(self, user_id, metric=EUCLIDEAN):
        metrics = {
            EUCLIDEAN: self.get_euclidean_distance,
            MANHATTAN: self.get_manhattan_distance,
            PEARSON: self.get_pearson_correlation,
        }

        distance_f = metrics[metric]

        similar_users = {}

        for similar_user_id in self.ratings:
            if similar_user_id == user_id:
                continue
            distance = distance_f(user_id, similar_user_id)
            if distance > 0:
                similar_users[similar_user_id] = distance

        return similar_users

        
movie_data = MovieData()

# Explore shared ratings

In [11]:
def explore_shared_ratings(movie_data):
    unique_user_ids = movie_data.get_unique_user_ids()

    n_pairs = 30
    samples = np.random.choice(unique_user_ids, size=(n_pairs, 2))

    for index, sample in enumerate(samples):
        user1_id = sample[0]
        user2_id = sample[1]

        num_movies_1 = len(movie_data.get_movies(user1_id))
        num_movies_2 = len(movie_data.get_movies(user2_id))

        num_shared_ratings = len(movie_data.get_shared_ratings(user1_id, user2_id))

        print 'pair %2d, user1 movies: %4d, user2 movies: %4d, shared movies: %3d' % (
            index + 1, num_movies_1, num_movies_2, num_shared_ratings)

        
explore_shared_ratings(movie_data)

pair  1, user1 movies:  111, user2 movies:  485, shared movies:  28
pair  2, user1 movies:   21, user2 movies:   20, shared movies:   0
pair  3, user1 movies:   63, user2 movies:   23, shared movies:   5
pair  4, user1 movies:  483, user2 movies:  159, shared movies:  87
pair  5, user1 movies:   22, user2 movies:   72, shared movies:   3
pair  6, user1 movies:   50, user2 movies:   20, shared movies:   0
pair  7, user1 movies:   22, user2 movies:  385, shared movies:   7
pair  8, user1 movies:  263, user2 movies:  129, shared movies:  26
pair  9, user1 movies:  300, user2 movies:   22, shared movies:   7
pair 10, user1 movies:   38, user2 movies:   61, shared movies:   0
pair 11, user1 movies:   87, user2 movies:   36, shared movies:   2
pair 12, user1 movies:  427, user2 movies:   79, shared movies:  31
pair 13, user1 movies:   20, user2 movies:  522, shared movies:   6
pair 14, user1 movies:  114, user2 movies:   87, shared movies:  14
pair 15, user1 movies:   28, user2 movies:   51,

We are looking at 30 random user pairs. We can notice how small on average is the intersection of the movies they rated (compared to the their total number of ratings).
It's not unusual to see zero intersection or just a couple of movies.

We could build a histogram of the distribution of number of shared movies if we generate a lot of random pairs.

# Explore distances

In [12]:
def explore_distances(movie_data):
    unique_user_ids = movie_data.get_unique_user_ids()

    n_pairs = 30
    samples = np.random.choice(unique_user_ids, size=(n_pairs, 2))

    for index, sample in enumerate(samples):
        user1_id = sample[0]
        user2_id = sample[1]

        num_shared_ratings = len(movie_data.get_shared_ratings(user1_id, user2_id))

        euclidean_distance = movie_data.get_euclidean_distance(user1_id, user2_id)
        manhattan_distance = movie_data.get_manhattan_distance(user1_id, user2_id)
        pearson_correlation = movie_data.get_pearson_correlation(user1_id, user2_id)

        print 'pair %2d, shared movies: %3d, euclidean: %.3f, manhattan: %.3f, pearson: %.3f' % (
            index + 1, num_shared_ratings, euclidean_distance, manhattan_distance, pearson_correlation)

        
explore_distances(movie_data)

pair  1, shared movies:  65, euclidean: 0.098, manhattan: 0.018, pearson: 0.210
pair  2, shared movies:   0, euclidean: 0.000, manhattan: 0.000, pearson: 0.000
pair  3, shared movies:  49, euclidean: 0.112, manhattan: 0.024, pearson: 0.201
pair  4, shared movies:   8, euclidean: 0.152, manhattan: 0.083, pearson: 0.123
pair  5, shared movies:   1, euclidean: 0.400, manhattan: 0.400, pearson: 0.000
pair  6, shared movies:   0, euclidean: 0.000, manhattan: 0.000, pearson: 0.000
pair  7, shared movies:   0, euclidean: 0.000, manhattan: 0.000, pearson: 0.000
pair  8, shared movies:   6, euclidean: 0.200, manhattan: 0.111, pearson: 0.100
pair  9, shared movies: 108, euclidean: 0.053, manhattan: 0.006, pearson: 0.088
pair 10, shared movies:  27, euclidean: 0.121, manhattan: 0.031, pearson: 0.023
pair 11, shared movies:  29, euclidean: 0.131, manhattan: 0.035, pearson: 0.257
pair 12, shared movies:   7, euclidean: 0.232, manhattan: 0.125, pearson: 0.490
pair 13, shared movies:   1, euclidean: 

Various distances (euclidean, manhattan, pearson correlation).

Other possible distances: Tantimoto, cosine.

Jaccard distance is not really applicable in this case since we have a range of ratings.

# Explore similar users

In [13]:
def explore_similar_users(movie_data):
    unique_user_ids = movie_data.get_unique_user_ids()

    n_users = 30
    user_ids = np.random.choice(unique_user_ids, size=n_users, replace=False)

    for index, user_id in enumerate(user_ids):
        similar_users = movie_data.get_similar_users(user_id)

        distances = similar_users.values()

        print 'user %3d, similar users: %d, max similarity: %.3f, mean similarity: %.3f, std similarity: %.3f' % (
            index + 1, len(similar_users), np.max(distances), np.mean(distances), np.std(distances))

        
explore_similar_users(movie_data)

user   1, similar users: 529, max similarity: 1.000, mean similarity: 0.407, std similarity: 0.234
user   2, similar users: 670, max similarity: 0.472, mean similarity: 0.143, std similarity: 0.069
user   3, similar users: 492, max similarity: 1.000, mean similarity: 0.379, std similarity: 0.227
user   4, similar users: 543, max similarity: 1.000, mean similarity: 0.408, std similarity: 0.217
user   5, similar users: 561, max similarity: 1.000, mean similarity: 0.368, std similarity: 0.228
user   6, similar users: 481, max similarity: 1.000, mean similarity: 0.508, std similarity: 0.280
user   7, similar users: 659, max similarity: 1.000, mean similarity: 0.186, std similarity: 0.117
user   8, similar users: 557, max similarity: 1.000, mean similarity: 0.357, std similarity: 0.215
user   9, similar users: 668, max similarity: 1.000, mean similarity: 0.188, std similarity: 0.114
user  10, similar users: 531, max similarity: 1.000, mean similarity: 0.390, std similarity: 0.229
user  11, 

Max similarity of 1.0 in most cases is probably an intersection of one movie.